In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
import subprocess
import numba

In [26]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
import subprocess
import numba

class WavePropagation():
    def __init__(self, nx, ny, nt, Lx, Ly, dt, filename, save_interval, t):
        self.nx = nx
        self.ny = ny
        self.nt = nt
        self.Lx = Lx
        self.Ly = Ly
        self.dt = dt
        self.u = np.zeros((nx, ny, nt), dtype=np.float32)
        self.filename = filename
        self.save_interval = save_interval
        self.t = t
        self.frame_count = 0  # Contador de fotogramas

    def initialize(self, sigma, x0, y0, A, epsilon, s, y_lamina):
        x = np.linspace(0, self.Lx, self.nx)
        y = np.linspace(0, self.Ly, self.ny)
        self.X, self.Y = np.meshgrid(x, y)
        self.sigma = sigma
        self.x0 = x0
        self.y0 = y0
        self.A = A
        self.epsilon = epsilon
        self.s = s
        self.y_lamina = y_lamina
        self.x_lamina1 = x0 - s / 2
        self.x_lamina2 = x0 + s / 2
        self.longitud_lamina = (self.Lx - s) / 2

    
    def propagate_wave(self, v, f, k):
        dx = self.Lx / (self.nx - 1)
        dy = self.Ly / (self.ny - 1)
        for t in range(0, self.nt - 1):
            for i in range(1, self.nx - 1):
                for j in range(1, self.ny - 1):
                    if not (self.y_lamina - dy / 2 < self.Y[i, j] < self.y_lamina + dy / 2 and (self.X[i, j] < self.x_lamina1 or self.X[i, j] > self.x_lamina2)):
                        self.u[i, j, t + 1] = (2 * self.u[i, j, t] - self.u[i, j, t - 1] + v ** 2 * self.dt ** 2 * (self.u[i + 1, j, t] - 2 * self.u[i, j, t] + self.u[i - 1, j, t]) / dx ** 2 + v ** 2 * self.dt ** 2 * (self.u[i, j + 1, t] - 2 * self.u[i, j, t] + self.u[i, j - 1, t]) / dy ** 2)
            self.u[0, :, t + 1] = 0
            self.u[-1, :, t + 1] = 0
            self.u[:, 0, t + 1] = 0
            self.u[:, -1, t + 1] = 0
            self.u[:, :, t + 1] += self.A * np.sin(-2 * np.pi * f * t * self.dt) * np.sin(k * (self.Y[:] - self.y0)) * np.sin(np.pi * (self.X[:]) / self.Lx) / np.sqrt((self.X[:] - self.x0) ** 2 + 5 * (self.Y[:] - self.y0) ** 2 + self.epsilon)
            if t % self.save_interval == 0:
                self.plot()

        self.create_video()
        self.cleanup_frames()

    def plot(self):
        u = self.u[:, :, self.frame_count]

        f = plt.figure(figsize=(5, 5), edgecolor='None')
        sp = f.add_subplot(1, 1, 1)
        plt.pcolor(u, cmap='rainbow', vmin=-self.A, vmax=self.A)
        plt.axis('tight')
        plt.xticks([])
        plt.yticks([])
        plt.savefig(f'frames/{self.filename}_{self.frame_count:04d}.png')
        plt.close(f)
        self.frame_count += 1

    def create_video(self):
        print(f"Creando video con {self.frame_count} fotogramas")
        command = [
            'ffmpeg', '-framerate', str((self.frame_count) / self.t), '-i', f'frames/{self.filename}_%04d.png',
            '-c:v', 'libx264', '-pix_fmt', 'yuv420p',
            f'{self.filename}_{self.nx}_iters_{self.nt}_frames_{self.frame_count}.mp4'
        ]
        subprocess.run(command)

    def cleanup_frames(self):
        for file in os.listdir('frames'):
            os.remove(os.path.join('frames', file))

# Parámetros de la simulación
nx, ny, nt = 100, 100, 4000
Lx, Ly = 10, 10
dt = 0.05
f = 5  # Frecuencia de la fuente
lambda_ = 5  # Longitud de onda
k = 2 * np.pi / lambda_  # Número de onda
v = lambda_ * f  # Velocidad de la onda

filename = f'wave_propagation_{lambda_}'
save_interval = 10
t = 30

# Inicializar la clase y los parámetros
wave = WavePropagation(nx, ny, nt, Lx, Ly, dt, filename, save_interval, t)
wave.initialize(sigma=0.2, x0=Lx/2, y0=0, A=2, epsilon=1e-12, s=0.25, y_lamina=Ly/5)

# Propagar la onda


wave.propagate_wave(v, f, k)

C:\Users\nicor\AppData\Local\Temp\ipykernel_4776\2413967081.py:45: RuntimeWarning: overflow encountered in cast
  self.u[i, j, t + 1] = (2 * self.u[i, j, t] - self.u[i, j, t - 1] + v ** 2 * self.dt ** 2 * (self.u[i + 1, j, t] - 2 * self.u[i, j, t] + self.u[i - 1, j, t]) / dx ** 2 + v ** 2 * self.dt ** 2 * (self.u[i, j + 1, t] - 2 * self.u[i, j, t] + self.u[i, j - 1, t]) / dy ** 2)
C:\Users\nicor\AppData\Local\Temp\ipykernel_4776\2413967081.py:45: RuntimeWarning: invalid value encountered in scalar add
  self.u[i, j, t + 1] = (2 * self.u[i, j, t] - self.u[i, j, t - 1] + v ** 2 * self.dt ** 2 * (self.u[i + 1, j, t] - 2 * self.u[i, j, t] + self.u[i - 1, j, t]) / dx ** 2 + v ** 2 * self.dt ** 2 * (self.u[i, j + 1, t] - 2 * self.u[i, j, t] + self.u[i, j - 1, t]) / dy ** 2)
C:\Users\nicor\AppData\Local\Temp\ipykernel_4776\2413967081.py:45: RuntimeWarning: invalid value encountered in scalar subtract
  self.u[i, j, t + 1] = (2 * self.u[i, j, t] - self.u[i, j, t - 1] + v ** 2 * self.dt ** 2 * 

Creando video con 400 fotogramas


In [8]:
# Parámetros de la simulación
nx, ny, nt = 100, 100, 2000
Lx, Ly = 10, 10
dt = 0.001
filename = 'wave_propagation'
save_interval = 10
t = 10

# Inicializar la clase y los parámetros
wave = WavePropagation(nx, ny, nt, Lx, Ly, dt, filename, save_interval, t)
wave.initialize(sigma=0.2, x0=Lx/2, y0=0, A=2, epsilon=1e-12, s=1, y_lamina=Ly/5)

# Propagar la onda
v = 1  # Velocidad de la onda
f = 1  # Frecuencia de la fuente
k = 2 * np.pi / Lx  # Número de onda
wave.propagate_wave(v, f, k)

Creando video con 2000 fotogramas
